In [4]:
import pandas as pd
import subprocess
import os
import shutil

# 复制原CSV文件为新的CSV文件
original_csv = 'text.csv'
modified_csv = 'textmodify.csv'
shutil.copyfile(original_csv, modified_csv)

# 读取修改后的CSV文件
df = pd.read_csv(modified_csv)

# 添加新的一栏 ifcombine，默认值为 0
df['ifcombine'] = 0

# 保存修改到同一个文件
df.to_csv(modified_csv, index=False)

In [17]:
import pandas as pd
import os
import subprocess
import shutil
modified_csv = 'textmodify.csv'
# 读取修改后的CSV文件
df = pd.read_csv(modified_csv)

# 创建结果文件夹和子文件夹
result_dir = "result"
result_pre_dir = os.path.join(result_dir, "result_pre")
os.makedirs(result_dir, exist_ok=True)
os.makedirs(result_pre_dir, exist_ok=True)

# 切割所有check值为1的视频并输出
for index, row in df.iterrows():
    video_name = row['name']
    start_time = float(row['start']) - 0.5  # 向前切割0.5秒
    end_time = float(row['end']) + 0.5  # 向后切割0.5秒

    if row['check'] == 1:
        if row['ifcombine'] == 0:
            # 如果不需要合并，直接切割并输出到result文件夹
            output_name = os.path.join(result_dir, f"{index + 1}.MP4")
            start_time = max(0, start_time)  # 确保起始时间不为负数
        elif row['ifcombine'] == 1:
            # 如果需要合并，切割并输出到result_pre文件夹
            output_name = os.path.join(result_pre_dir, f"{index + 1}_1.MP4")

        if os.path.exists(video_name):
            subprocess.run([
                'ffmpeg', 
                '-i', video_name, 
                '-ss', str(start_time), 
                '-to', str(end_time), 
                '-c', 'copy', 
                output_name
            ])
        else:
            print(f"File {video_name} does not exist")

# 处理result_pre目录下的视频并合并
pre_files = sorted(os.listdir(result_pre_dir))
current_start = None
current_end = None
current_video_name = None
merge_index = 0
for filename in pre_files:
    if filename.endswith("_1.MP4"):
        video_name = os.path.join(result_pre_dir, filename)
        id_number = filename.split("_")[0]
        if current_start is None:
            current_start = id_number
            current_video_name = video_name
            start_time = float(df.loc[int(id_number)-1, 'start']) - 0.5  # 向前切割0.5秒
        current_end = id_number
        end_time = float(df.loc[int(id_number)-1, 'end']) + 0.5  # 向后切割0.5秒

    if current_start is not None and current_end is not None and current_start != current_end:
        output_name = os.path.join(result_dir, f"{current_start}.MP4")
        if os.path.exists(current_video_name):
            subprocess.run([
                'ffmpeg', 
                '-i', f"concat:{current_video_name}|{os.path.join(result_pre_dir, current_end + '_1.MP4')}", 
                '-ss', str(start_time),
                '-to', str(end_time),
                '-c', 'copy', 
                output_name
            ])
        else:
            print(f"File {current_video_name} does not exist")
        merge_index += 1
        current_start = None
        current_end = None
